# Preparation

In [ ]:
import os
import datetime
import geopandas as gpd
import pandas as pd

In [ ]:
# read inputs from spreadsheet
working_dr = os.getcwd()
inputs_dr = os.path.join(working_dr, 'inputs.csv')
local_images_dr = os.path.join(working_dr, 'local_data')
inputs_df = pd.read_csv(inputs_dr)

In [ ]:
# choose a season

# by index
index = 0
row = inputs_df.iloc[index]

# # by name
# name = '2024_winter'
# row = inputs_df.loc[inputs_df['season'] == name].iloc[0]

season = row['season']
project_area_dr = row['project_area']
field_area_dr = row['field_area']
PET_area_dr = row['PET_area']
start_date = row['start_date']
end_date = row['end_date']

project_gdf = gpd.read_file(project_area_dr)
field_gdf = gpd.read_file(field_area_dr)
PET_gdf = gpd.read_file(PET_area_dr)

# Products
------------------------------

## [geeSEBAL](docs/sebal.md)
Surface Energy Balance Algorithm for Land (SEBAL), a model for estimating evapotranspiration (ET) from energy balance equation.\
geeSEBAL is an open-source implementation of (SEBAL) using Google Earth Engine (GEE).
### outputs
* ETa = actual evapotranspiration
* NDVI = Normalized Difference Vegetation Index
* RGB = red,green,blue image

In [ ]:
import ee
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
# creat area of interest for google earth engine
bbox = project_gdf.total_bounds.tolist()
aoi = ee.Geometry.Rectangle(bbox)
geojson = field_gdf.__geo_interface__
aoi_gee = ee.FeatureCollection(geojson['features'])

In [ ]:
from products import Image_gee
aoi = 1
hot_cold_limits = {'NDVI_cold':1,
									'Ts_cold':1,
									'NDVI_hot':1,
									'Ts_hot':1}

result_imgs = Image_gee(start_date, end_date, aoi, hot_cold_limits)

In [ ]:
# export
from funcs import export_to_drive, export_gee_tifs_localy

output_dr = os.path.join(working_dr, 'local_data', 'products_outputs', season, 'geeSEBAL')

# # export to drive (gee)
# for img in result_imgs:
#     # get img date
#     date = img.date_string
#     # export tif (to drive)
#     export_to_drive(img, date, os.path.join('monitoring_output_geeSEBAL', f'season_{season}'), aoi)

# export localy (gee)
for img in result_imgs:
  try:
    date = img.date_string
    export_gee_tifs_localy(img.image, 'RGB', 'rgb', output_dr, date, aoi)
    export_gee_tifs_localy(img.image, 'ET_24h', 'eta', output_dr, date, aoi)
    export_gee_tifs_localy(img.image, 'NDVI', 'ndvi', output_dr, date, aoi)
  except Exception as error:
    print(img.landsat_version)
    print(img.date_string)
    print(f"Exception = {error}")

## [SEBAL](docs/sebal.md)
Surface Energy Balance Algorithm for Land (SEBAL), a model for estimating evapotranspiration (ET) from energy balance equation.
### outputs
* ETa = actual evapotranspiration
* NDVI = Normalized Difference Vegetation Index
* RGB = red,green,blue image

In [ ]:
# for local images
import glob
from products import Image_local

output_dr = os.path.join(working_dr, 'local_data', 'products_outputs', season, 'SEBAL')

local_ls_images = [os.path.split(i)[0] for i in glob.glob(os.path.join(local_images_dr, 'landsat', 'images', '*','*/'))]
for local_image in local_ls_images:
  landsat_img = Image_local(local_image, local_images_dr, output_dr,
                            NDVI_cold=1,
                            Ts_cold=1,
                            NDVI_hot=1,
                            Ts_hot=1)

## [Growth Monitoring](docs/growth.md)

Using Sentinel Application Platform (SNAP) command line feature to estimate leaf area index (LAI).
### outputs
* LAI = leaf area index
* NDVI = Normalized Difference Vegetation Index

In [ ]:
from products import s2_biophysical_processor

gpt = "/home/msalah/esa-snap/bin/gpt"		# path to gpt executable
s2_dr = os.path.join(working_dr, 'local_data', 'sentinel_2')
output_dr = os.path.join(working_dr, 'local_data', 'products_outputs', season, 'growth')
s2_biophysical_processor(gpt, s2_dr, output_dr, project_gdf)

## [WaPOR data](https://www.fao.org/in-action/remote-sensing-for-water-productivity/en)
FAO’s portal to monitor Water Productivity through Open access of Remotely (WaPOR) sensed derived data, monitors and reports on agriculture water productivity over Africa and the Near East.

In [ ]:
from products import download_WaPOR

output_dr = os.path.join(working_dr, 'local_data', 'products_outputs', season, 'WaPOR')
period = [start_date, end_date]
overview = "NONE"
variables = ["L2-AETI-D", "L2-NPP-D"]
region = project_area_dr
download_WaPOR(region, variables, period, output_dr, overview)
variables = ["L1-RET-E"]
region = PET_area_dr
download_WaPOR(region, variables, period, output_dr, overview)